In [ ]:
from zipfile import ZipFile
import os

# Path where you uploaded your zip
zip_path = "/content/fl_manufacturers.zip"

# Extract to /content/dataset
extract_path = "/content/dataset"

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")
print("Folders inside dataset:", os.listdir(extract_path))


Extraction complete!
Folders inside dataset: ['fl_manufacturers']


In [ ]:
# --- Step 1: Setup ---
!pip install torch torchvision torchaudio --quiet

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --- Step 2: Data transforms ---
data_dir = "/content/dataset/fl_manufacturers"  # <-- upload your dataset folder here

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# --- Step 3: Load dataset ---
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

class_names = train_dataset.classes
print("Classes:", class_names)

# --- Step 4: Load pretrained model ---
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# --- Step 5: Train model ---
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Acc: {100*correct/total:.2f}%")

# --- Step 6: Save model ---
torch.save(model.state_dict(), "aircraft_model.pth")

# --- Step 7: Prediction function ---
def predict_image(img_path):
    model.eval()
    image = Image.open(img_path).convert("RGB")
    transform = transform_val
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return class_names[predicted.item()]

# --- Step 8: Test with new image ---
# Copy-paste your image path here
#test_image_path = "/content/test_aircraft.jpg"  # <- replace with your uploaded image path
#print("Prediction:", predict_image(test_image_path))


Using device: cuda
Classes: ['AIRBUS', 'ATR', 'B-2_Bomber', 'B-52_Bomber', 'BOEING', 'BOEING_DEFENCE', 'F-16', 'F-22', 'F-35']


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 89.3MB/s]


Epoch 1/20, Loss: 0.7550, Acc: 73.82%
Epoch 2/20, Loss: 0.2266, Acc: 92.12%
Epoch 3/20, Loss: 0.1323, Acc: 95.90%
Epoch 4/20, Loss: 0.0839, Acc: 97.26%
Epoch 5/20, Loss: 0.0508, Acc: 98.63%
Epoch 6/20, Loss: 0.0413, Acc: 98.92%
Epoch 7/20, Loss: 0.0538, Acc: 98.77%
Epoch 8/20, Loss: 0.0449, Acc: 98.82%
Epoch 9/20, Loss: 0.0291, Acc: 99.10%
Epoch 10/20, Loss: 0.0209, Acc: 99.48%
Epoch 11/20, Loss: 0.0339, Acc: 99.10%
Epoch 12/20, Loss: 0.0326, Acc: 98.96%
Epoch 13/20, Loss: 0.0352, Acc: 99.06%
Epoch 14/20, Loss: 0.0183, Acc: 99.58%
Epoch 15/20, Loss: 0.0274, Acc: 99.20%
Epoch 16/20, Loss: 0.0178, Acc: 99.53%
Epoch 17/20, Loss: 0.0245, Acc: 99.06%
Epoch 18/20, Loss: 0.0417, Acc: 98.49%
Epoch 19/20, Loss: 0.0146, Acc: 99.62%
Epoch 20/20, Loss: 0.0392, Acc: 98.82%


In [ ]:
# Copy-paste your image path here
#test_image_path = "/content/download.jpg"  # <- replace with your uploaded image path
#print("Prediction:", predict_image(test_image_path))

In [ ]:
!pip install ultralytics torch torchvision opencv-python --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()  # Choose your files


Saving 3620476-hd_1920_1080_25fps.mp4 to 3620476-hd_1920_1080_25fps.mp4


In [ ]:
import torch
from torchvision import models, transforms
from ultralytics import YOLO
from PIL import Image
import cv2

# Load YOLOv8 pre-trained (for airplane detection)
yolo_model = YOLO("yolov8n.pt")

# Your class labels
class_names = ["AIRBUS", "ATR", "B-2_Bomber", "B-52_Bomber", "BOEING", "BOEING_DEFENCE", "F-16", "F-22", "F-35"]

# Load your classifier
classifier = models.resnet18(pretrained=False)
classifier.fc = torch.nn.Linear(classifier.fc.in_features, len(class_names))
classifier.load_state_dict(torch.load("aircraft_model.pth", map_location="cpu"))
classifier.eval()

# Preprocessing for classifier
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def classify_crop(image):
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    img = transform(img).unsqueeze(0)
    with torch.no_grad():
        outputs = classifier(img)
        _, pred = torch.max(outputs, 1)
    return class_names[pred.item()]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
video_path = "/content/b2.mp4"   # uploaded video file
output_path = "output.mp4"

cap = cv2.VideoCapture(video_path)

out = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*'mp4v'),
    cap.get(cv2.CAP_PROP_FPS),
    (int(cap.get(3)), int(cap.get(4)))
)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = yolo_model(frame, conf=0.3, verbose=False)

    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])  # COCO class id
            if cls == 4:  # "aeroplane"
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                crop = frame[y1:y2, x1:x2]
                if crop.size > 0:
                    label = classify_crop(crop)
                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                    cv2.putText(frame, label, (x1, y1-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Done! Processed video saved as", output_path)


✅ Done! Processed video saved as output.mp4


In [ ]:
from google.colab import files
files.download("output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>